<a href="https://colab.research.google.com/github/sunspai1/Capstone-Project/blob/main/Feature_Extraction_from_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%writefile 1.txt
This is a story about cats
our feline pets
Cats are furry animals

Writing 1.txt


In [4]:
%%writefile 2.txt
This story is about surfing
Catching waves is fun
Surfing is a popular water sport

Writing 2.txt


#Build a vocabulary
The goal here is to build a numerical array from all the words that appear in every document. Later we'll create instances (vectors) for each individual document.

In [5]:
vocab = {}
i=1

with open('1.txt') as f:
  x=f.read().lower().split()

for word in x:
  if word in vocab:
    continue
  else:
    vocab[word] = i
    i += 1
print(vocab)


{'this': 1, 'is': 2, 'a': 3, 'story': 4, 'about': 5, 'cats': 6, 'our': 7, 'feline': 8, 'pets': 9, 'are': 10, 'furry': 11, 'animals': 12}


In [6]:
vocab = {}
i=1

with open('2.txt') as f:
  x=f.read().lower().split()

for word in x:
  if word in vocab:
    continue
  else:
    vocab[word] = i
    i += 1
print(vocab)

{'this': 1, 'story': 2, 'is': 3, 'about': 4, 'surfing': 5, 'catching': 6, 'waves': 7, 'fun': 8, 'a': 9, 'popular': 10, 'water': 11, 'sport': 12}


#**Feature** Extraction

In [7]:
# Perform imports and load the dataset:
import numpy as np
import pandas as pd

In [8]:
from google.colab import files
data_to_load = files.upload()

Saving smsspamcollection.tsv to smsspamcollection.tsv


In [9]:
import io
df = pd.read_csv(io.BytesIO(data_to_load['smsspamcollection.tsv']),sep='\t')

In [10]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [11]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [12]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
x = df['message']
y= df['label']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.33, random_state=42)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(3733, 7082)

In [16]:
X_train.shape

(3733,)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vectorizer = TfidfVectorizer()

In [19]:
X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(3733, 7082)

In [20]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

**Build** **a** **Pipeline**

In [21]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC()),])

In [22]:
# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [23]:
# Form a prediction set
predictions = text_clf.predict(X_test)

In [24]:
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [25]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [26]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.989668297988037
